## Part 1

In [2]:
import numpy as np
import pandas as pd
!pip install geopy
!pip install folium

In [3]:
tab=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
df=tab[0]
df.columns=[' PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


As we want only rows where Borough should not be (not assigned)  we  will only select dataframe where    Borough!='Not Assigned'

In [5]:
df_neigh=df[df['Borough']!='Not assigned']
df_neigh.reset_index(inplace=True)
df_neigh.drop(columns='index',inplace=True)
df_neigh.head(10)

C:\Users\digvi\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Part2

We will check whether Duplicater values of Postal Code exists or not

In [6]:
bool_series = df_neigh.iloc[:,0].duplicated(keep = False) 
df_neigh[bool_series]

,PostalCode,Borough,Neighborhood


As we need to drop Not Assign rows we will compare Neighbour = Not assigned or not

In [7]:
df_neigh[df_neigh['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [8]:
df_neigh.shape

(103, 3)

In [9]:
geo_data=pd.read_csv('Geospatial_Coordinates.csv')


In [10]:
geo_data[0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
lat=[]
long=[]
for index , row in geo_data.iterrows() :
    for j, r in  df_neigh.iterrows() :
        if row[0]==r[0] :
            lat.append(row[1])
            long.append(row[2])
            continue
            

In [12]:
df_neigh['Latitude']= lat
df_neigh['Longitude']= long

C:\Users\digvi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\digvi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
df_neigh.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


# Part 3

### ploting map of all neighbourhood 

In [14]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neigh['Latitude'], df_neigh['Longitude'], df_neigh['Borough'], df_neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Verifing extracted location data is correct or not

In [53]:
df_neigh[df_neigh['Neighborhood']=='Central Bay Street']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259


In [16]:
geo_data[geo_data['Postal Code']=='M5G']

,Postal Code,Latitude,Longitude
57,M5G,43.657952,-79.387383


makeing DataFrame df_toronto where borough are only Toronto

In [17]:
df_toronto=df_neigh[df_neigh['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop(columns='index',inplace=True)
df_toronto.head(10)

C:\Users\digvi\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975
5,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
6,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
7,M6G,Downtown Toronto,Christie,43.753259,-79.329656
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944


In [18]:
df_toronto=df_toronto[['Borough','Neighborhood','Latitude','Longitude']]
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,Downtown Toronto,St. James Town,43.799525,-79.318389
4,East Toronto,The Beaches,43.786947,-79.385975


In [19]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Ploting graph for toronto borough only 

In [20]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
CLIENT_ID = '10MCESV5DYWAO0A3B1PMN5PEPGILFKXYYXCSUMTA3AHTLQAH' # your Foursquare ID
CLIENT_SECRET = 'GJ54RTFWQCODJWSTWWCBMTZNFMJANXESGV5ZV02R14U0JEIT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#while sharing this book make this code hide
radius=500
LIMIT=100


In [22]:
df_toronto['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [23]:
import json 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [24]:
df_toronto.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

# Exploring only one borough of toronto

In [25]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


In [26]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?&client_id=10MCESV5DYWAO0A3B1PMN5PEPGILFKXYYXCSUMTA3AHTLQAH&client_secret=GJ54RTFWQCODJWSTWWCBMTZNFMJANXESGV5ZV02R14U0JEIT&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f6f16f8da9f742ef0b60d58'},
 'response': {'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.76678992471017,
          'lng': -79.1911511887

In [28]:
ven=results["response"]['groups'][0]['items']
nearby_venues = json_normalize(ven)

In [29]:
nearby_venues.iloc[0:1]

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4beee041e24d20a1cd857314-0,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",4beee041e24d20a1cd857314,4374 KINGSTON RD,CA,Scarborough,Canada,Kingston & Lawrence,408,"[4374 KINGSTON RD (Kingston & Lawrence), Scarb...","[{'label': 'display', 'lat': 43.76678992471017...",43.76679,-79.191151,M1E 2M8,ON,RBC Royal Bank,0,[]


In [30]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.iloc[0:1]

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,RBC Royal Bank,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",43.76679,-79.191151


In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


# declaring function for making api calls for every borough

In [33]:


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Exploring data for all borough location

In [34]:

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(748, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
"Brockton, Parkdale Village, Exhibition Place",37,37,37,37,37,37
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",4,4,4,4,4,4
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,5,5,5,5,5,5
Christie,2,2,2,2,2,2
Church and Wellesley,6,6,6,6,6,6
"Commerce Court, Victoria Hotel",2,2,2,2,2,2
Davisville,3,3,3,3,3,3


In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.



# Analyze each dataframe


In [38]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_grouped =toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.027027,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.054054,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.027027,0.0000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
6,Church and Wellesley,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.500000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.0,0.054054,0.00,0.040541,0.013514,0.0000,0.00


In [40]:
toronto_onehot.shape

(748, 206)

In [41]:
toronto_grouped.shape

(39, 206)

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                             venue  freq
0              Martial Arts School   1.0
1                      Yoga Studio   0.0
2  Molecular Gastronomy Restaurant   0.0
3                        Locksmith   0.0
4                           Lounge   0.0


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.08
1            Gastropub  0.05
2               Bakery  0.05
3              Brewery  0.05
4  American Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                        venue  freq
0                        Park  0.25
1                 Pizza Place  0.25
2              Sandwich Place  0.25
3                    Bus Line  0.25
4  Modern European Restaurant  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
           venue  freq
0           Park  0.06
1  Auto Workshop  0.06
2       

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts School,Women's Store,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Café,Yoga Studio,Cheese Shop,Fish Market,Bookstore
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Sandwich Place,Bus Line,Women's Store,Diner,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Brewery,Garden,Recording Studio,Light Rail Station,Fast Food Restaurant,Auto Workshop,Pizza Place,Farmers Market,Burrito Place,Park
4,Central Bay Street,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# making a k-means model

In [45]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 1, 1, 1, 1, 2, 1, 4, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4,
       1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1])

In [46]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,1,Medical Center,Breakfast Spot,Electronics Store,Intersection,Restaurant,Bank,Rental Car Location,Mexican Restaurant,Distribution Center,Eastern European Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,1,Bakery,Bank,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Gas Station,Dog Run,Distribution Center
2,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,1,Café,College Stadium,General Entertainment,Skating Rink,Women's Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,Downtown Toronto,St. James Town,43.799525,-79.318389,1,Fast Food Restaurant,Gym,Pharmacy,Indian Restaurant,Breakfast Spot,Chinese Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Grocery Store
4,East Toronto,The Beaches,43.786947,-79.385975,1,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Distribution Center,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


# ploting graph for each cluster

In [47]:
import matplotlib.cm as cm
import matplotlib.colors as colors



# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Berczy Park,Martial Arts School,Women's Store,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Medical Center,Breakfast Spot,Electronics Store,Intersection,Restaurant,Bank,Rental Car Location,Mexican Restaurant,Distribution Center,Eastern European Restaurant
1,"Queen's Park, Ontario Provincial Government",Bakery,Bank,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Gas Station,Dog Run,Distribution Center
2,"Garden District, Ryerson",Café,College Stadium,General Entertainment,Skating Rink,Women's Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,St. James Town,Fast Food Restaurant,Gym,Pharmacy,Indian Restaurant,Breakfast Spot,Chinese Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Grocery Store
4,The Beaches,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Distribution Center,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
6,Central Bay Street,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,"Dufferin, Dovercourt Village",Grocery Store,Hotel,Bank,Shopping Mall,Park,Coworking Space,Distribution Center,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
10,"Harbourfront East, Union Station, Toronto Islands",Skating Rink,Curling Ice,Beer Store,Athletics & Sports,Dance Studio,Park,Cuban Restaurant,Creperie,Farmers Market,Falafel Restaurant
11,"Little Portugal, Trinity",Trail,Health Food Store,Pub,Women's Store,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
12,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Furniture / Home Store,Ice Cream Shop,Lounge,Pizza Place,Liquor Store


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
8,"Richmond, Adelaide, King",Park,Airport,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
24,"The Annex, North Midtown, Yorkville",Park,Pool,Women's Store,Airport Food Court,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,Construction & Landscaping,Baseball Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
36,"First Canadian Place, Underground city",Baseball Field,Women's Store,Field,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,"Commerce Court, Victoria Hotel",Playground,Trail,Women's Store,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,"North Toronto West, Lawrence Park",Field,Trail,Hockey Arena,Tennis Court,Diner,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
